[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 6 - Amazon Athena

[Wrangler](https://github.com/awslabs/aws-data-wrangler) has two ways to run queries on Athena and fetch the result as a DataFrame:

- **ctas_approach=True** (Default)

    Wraps the query with a CTAS and then reads the table data as parquet directly from s3.
    
    `PROS`: Faster and can handle some level of nested types.
    
    `CONS`: Requires create/delete table permissions on Glue and Does not support timestamp with time zone (A temporary table will be created and then deleted immediately).

- **ctas_approach=False**

    Does a regular query on Athena and parse the regular CSV result on s3.
    
    `PROS`: Does not require create/delete table permissions on Glue and supports timestamp with time zone.
    
    `CONS`: Slower (But stills faster than other libraries that uses the regular Athena API) and does not handle nested types at all.

In [1]:
import awswrangler as wr

## Enter your bucket name:

In [2]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

 ············


## Checking/Creating Glue Catalog Databases

In [3]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

### Creating a Parquet Table from the NOAA's CSV files

[Reference](https://registry.opendata.aws/noaa-ghcn/)

In [4]:
cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]

df = wr.s3.read_csv(
    path="s3://noaa-ghcn-pds/csv/189",
    names=cols,
    parse_dates=["dt", "obs_time"])  # Read 10 files from the 1890 decade (~1GB)

df

,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ASN00070200,1890-01-01,PRCP,0,NaN,NaN,a,NaN
1,SF000782720,1890-01-01,PRCP,0,NaN,NaN,I,NaN
2,CA005022790,1890-01-01,TMAX,-222,NaN,NaN,C,NaN
3,CA005022790,1890-01-01,TMIN,-261,NaN,NaN,C,NaN
4,CA005022790,1890-01-01,PRCP,0,NaN,NaN,C,NaN
...,...,...,...,...,...,...,...,...
29240014,USC00181790,1899-12-31,PRCP,0,P,NaN,6,1830
29240015,ASN00061000,1899-12-31,PRCP,0,NaN,NaN,a,NaN
29240016,ASN00040284,1899-12-31,PRCP,0,NaN,NaN,a,NaN
29240017,ASN00048117,1899-12-31,PRCP,0,NaN,NaN,a,NaN


In [5]:
wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database="awswrangler_test",
    table="noaa"
);

In [6]:
wr.catalog.table(database="awswrangler_test", table="noaa")

,Column Name,Type,Partition,Comment
0,id,string,False,
1,dt,timestamp,False,
2,element,string,False,
3,value,bigint,False,
4,m_flag,string,False,
5,q_flag,string,False,
6,s_flag,string,False,
7,obs_time,string,False,


## Default reading (ctas_approach=True) 30x faster!

In [7]:
%%time

wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test")

CPU times: user 21.6 s, sys: 5.52 s, total: 27.2 s
Wall time: 48.8 s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,USC00047738,1898-01-13,PRCP,5,<NA>,<NA>,0,<NA>
1,USC00047738,1898-01-13,SNOW,0,<NA>,<NA>,0,<NA>
2,USC00047738,1898-01-13,SNWD,0,<NA>,<NA>,0,<NA>
3,USC00047738,1898-01-13,TMAX,122,<NA>,<NA>,0,<NA>
4,USC00047738,1898-01-13,TMIN,28,<NA>,<NA>,0,<NA>
...,...,...,...,...,...,...,...,...
29240014,USC00181790,1899-12-31,PRCP,0,P,<NA>,6,1830
29240015,ASN00061000,1899-12-31,PRCP,0,<NA>,<NA>,a,<NA>
29240016,ASN00040284,1899-12-31,PRCP,0,<NA>,<NA>,a,<NA>
29240017,ASN00048117,1899-12-31,PRCP,0,<NA>,<NA>,a,<NA>


## Reading with ctas_approach=False

In [8]:
%%time

wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test", ctas_approach=False)

CPU times: user 9min 6s, sys: 6.9 s, total: 9min 13s
Wall time: 11min 24s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,ASN00070200,1890-01-01,PRCP,0,<NA>,<NA>,a,<NA>
1,SF000782720,1890-01-01,PRCP,0,<NA>,<NA>,I,<NA>
2,CA005022790,1890-01-01,TMAX,-222,<NA>,<NA>,C,<NA>
3,CA005022790,1890-01-01,TMIN,-261,<NA>,<NA>,C,<NA>
4,CA005022790,1890-01-01,PRCP,0,<NA>,<NA>,C,<NA>
...,...,...,...,...,...,...,...,...
29240014,USC00303773,1899-12-31,TMIN,-156,<NA>,<NA>,6,<NA>
29240015,USC00303773,1899-12-31,PRCP,0,<NA>,<NA>,6,<NA>
29240016,USC00303773,1899-12-31,SNOW,0,<NA>,<NA>,6,<NA>
29240017,USC00165090,1899-12-31,TMAX,100,<NA>,<NA>,6,<NA>


## Using categories to speed up and save memory!

In [9]:
%%time

wr.athena.read_sql_query("SELECT * FROM noaa", database="awswrangler_test", categories=["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"])

CPU times: user 4.54 s, sys: 2.05 s, total: 6.59 s
Wall time: 28.2 s


,id,dt,element,value,m_flag,q_flag,s_flag,obs_time
0,USC00043111,1893-01-15,PRCP,33,NaN,NaN,6,NaN
1,ASN00015525,1893-01-15,PRCP,0,NaN,NaN,a,NaN
2,ASN00051002,1893-01-15,PRCP,0,NaN,NaN,a,NaN
3,USC00204502,1893-01-15,PRCP,0,P,NaN,6,NaN
4,USC00320720,1893-01-15,TMAX,-144,NaN,NaN,6,NaN
...,...,...,...,...,...,...,...,...
29240014,USC00395481,1899-12-31,TMIN,-250,NaN,NaN,6,NaN
29240015,USC00395481,1899-12-31,PRCP,0,P,NaN,6,NaN
29240016,USC00395481,1899-12-31,SNOW,0,NaN,NaN,6,NaN
29240017,ASN00063055,1899-12-31,PRCP,0,NaN,NaN,a,NaN


## Batching (Good for restricted memory environments)

In [10]:
dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    ctas_approach=True,
    chunksize=True  # Chunksize calculated automatically for ctas_approach.
)

for df in dfs:  # Batching
    print(len(df.index))

5120
185344
2403328
10183680
9361107
7101440


In [12]:
dfs = wr.athena.read_sql_query(
    "SELECT * FROM noaa",
    database="awswrangler_test",
    ctas_approach=False,
    chunksize=10_000_000
)

for df in dfs:  # Batching
    print(len(df.index))

10000000
10000000
9240019


## Cleaning Up S3

In [13]:
wr.s3.delete_objects(path)

## Delete table

In [14]:
wr.catalog.delete_table_if_exists(database="awswrangler_test", table="noaa");

## Delete Database

In [15]:
wr.catalog.delete_database('awswrangler_test')